In [71]:
import json
from tqdm import tqdm

In [72]:
with open('conversations.json', 'r') as f:
    conversations = json.load(f)

In [83]:
def get_msg(convo):
    mapping = convo['mapping']
    user_msgs = []
    
    for node in mapping.keys():
        if mapping[node]['message']:
            if 'parts' in mapping[node]['message']['content'].keys() and mapping[node]['message']['author']['role'] == 'user':
                if isinstance(mapping[node]['message']['content']['parts'][0], str):
                    user_msgs.append(mapping[node]['message']['content']['parts'])
    return user_msgs

In [84]:
all_user_msgs = []
for convo in tqdm(conversations):
    all_user_msgs.extend(get_msg(convo))

len(all_user_msgs)

100%|██████████| 3261/3261 [00:00<00:00, 8199.67it/s]


14157

In [ ]:
from openai import OpenAI
import os 
from dotenv import load_dotenv
import tiktoken

load_dotenv()

MAX_CONTEXT_LENGTH = 8192
EMBEDDING_MODEL = "text-embedding-3-small"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
embeddings = []

for t in tqdm(all_user_msgs):
    string_t = t[0]
    embedding = tiktoken.encoding_for_model(EMBEDDING_MODEL).encode(string_t)
    num_tokens = len(embedding)
    try:
        if num_tokens > MAX_CONTEXT_LENGTH:
            string_t = string_t[-MAX_CONTEXT_LENGTH:]
        embedding = client.embeddings.create(input=string_t, model=EMBEDDING_MODEL).data[0].embedding
        embeddings.append(embedding)
    except Exception as e:
        print(e)
        break




 18%|█▊        | 2562/13947 [50:39<3:45:06,  1.19s/it]


KeyboardInterrupt: 